In [ ]:
# --------------------------
# 1. 读入数据 (Read data)
# --------------------------
# 修改为你的文件路径/工作表名称 (Modify to your file path/sheet name)
df = pd.read_excel("367EHRI Sensitivity Analysis.xlsx", sheet_name='Sheet1')

# --------------------------
# 2. 计算人体健康端点 P_HH (Calculate human health endpoint P_HH)
# --------------------------
# df["P_HH"] = np.maximum(df["P_C"], df["P_NC"])

# --------------------------
# 3. 定义 TEHR 计算函数 (Define TEHR calculation function)
# --------------------------
def calc_tehr(df, w_eco: float, w_hh: float, col_name: str) -> None:
    """向 DataFrame 添加一列 TEHR = w_eco*P_E + w_hh*P_HH"""
    """Add a column TEHR = w_eco*P_E + w_hh*P_HH to DataFrame"""
    df[col_name] = w_eco * df["ERA_Max"] + w_hh * df["HHRA_Max"]

# (i) 基准权重 0.42 : 0.58（UNEP-GLAM）((i) Baseline weights 0.42 : 0.58 (UNEP-GLAM))
w_hh_base = 0.58
calc_tehr(df, w_eco=1 - w_hh_base, w_hh=w_hh_base, col_name="TEHR_base")

# (ii) ReCiPe 40 / 40（→ 0.50 / 0.50）((ii) ReCiPe 40 / 40 (→ 0.50 / 0.50))
calc_tehr(df, w_eco=0.50, w_hh=0.50, col_name="TEHR_recipe")

# (iii) ±20 % 敏感性分析（保持 w_eco + w_hh = 1）((iii) ±20 % sensitivity analysis (maintain w_eco + w_hh = 1))
w_hh_up   = min(1.0, w_hh_base * 1.20)
w_hh_down = w_hh_base * 0.80
calc_tehr(df, w_eco=1 - w_hh_up,   w_hh=w_hh_up,   col_name="TEHR_up20")
calc_tehr(df, w_eco=1 - w_hh_down, w_hh=w_hh_down, col_name="TEHR_down20")

# --------------------------
# 4. 排名与 Kendall-τ 比较 (Ranking and Kendall-τ comparison)
# --------------------------
def kendall_tau(a, b):
    """返回 Kendall τ 相关系数（忽略 NA）"""
    """Return Kendall τ correlation coefficient (ignoring NA)"""
    return kendalltau(a, b, nan_policy="omit").correlation

rank_cols = ["TEHR_base", "TEHR_recipe", "TEHR_up20", "TEHR_down20"]
# 将得分越大排名越靠前（ascending=False）(Higher scores rank higher (ascending=False))
ranks = df[rank_cols].rank(ascending=False, method="min")

tau_recipe = kendall_tau(ranks["TEHR_base"], ranks["TEHR_recipe"])
tau_up20   = kendall_tau(ranks["TEHR_base"], ranks["TEHR_up20"])
tau_down20 = kendall_tau(ranks["TEHR_base"], ranks["TEHR_down20"])

print("Kendall τ 与基准比较")  # (Kendall τ compared to baseline)
print(f"  ReCiPe   : {tau_recipe:.3f}")
print(f"  +20 % w_HH: {tau_up20:.3f}")
print(f"  –20 % w_HH: {tau_down20:.3f}")

# --------------------------
# 5. 输出结果 (Output results)
# --------------------------
# 结果汇总表 (Results summary table)
summary = pd.DataFrame({
    "Scenario": ["ReCiPe", "+20 % w_HH", "–20 % w_HH"],
    "Kendall_tau_vs_base": [tau_recipe, tau_up20, tau_down20],
    "Material_rank_shift": [abs(1 - tau_recipe) > 0.05,
                            abs(1 - tau_up20)   > 0.05,
                            abs(1 - tau_down20) > 0.05]
})
print("\nRank-shift summary")  # (排名变化摘要)
print(summary)

# 保存带有全部 TEHR 分数和排名的 Excel (Save Excel with all TEHR scores and ranks)
df_out = pd.concat([df, ranks.add_suffix("_rank")], axis=1)
df_out.to_excel("tehr_results.xlsx", index=False)


Kendall τ 与基准比较
  ReCiPe   : 0.932
  +20 % w_HH: 0.913
  –20 % w_HH: 0.899

Rank-shift summary
     Scenario  Kendall_tau_vs_base  Material_rank_shift
0      ReCiPe             0.931746                 True
1  +20 % w_HH             0.912568                 True
2  –20 % w_HH             0.898661                 True


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kendalltau

# ---------------------------------------------------------------------
# 参数区 (Parameter section)
EXCEL_FILE = "367EHRI Sensitivity Analysis.xlsx"   # 输入文件 (Input file)
SHEET_NAME = 'Sheet1'                 # 工作表索引/名称 (Worksheet index/name)
N_SIM = 1000                    # Monte‑Carlo 迭代次数 (Monte Carlo iterations)
W_HH_BASE = 0.58                # 基准人体健康权重 (UNEP‑GLAM) (Baseline human health weight (UNEP-GLAM))
PERTURB_FRAC = 0.20             # 浮动幅度 ±20 % (Perturbation magnitude ±20%)
# ---------------------------------------------------------------------

# 1. 读取数据 (Read data) -----------------------------------------------------------
df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME)

# # 2. 计算保守人体健康概率 (Calculate conservative human health probability) ---------------------------------------------
# df["P_HH"] = np.maximum(df["P_C"], df["P_NC"])

# 3. 辅助函数：计算并附加 EHRI 列 (Helper function: calculate and append EHRI column) -------------------------------------

def add_tehr(df, w_eco: float, w_hh: float, col: str) -> None:
    df[col] = w_eco * df["ERA_Max"] + w_hh * df["HHRA_Max"]

# 3a. 基准 TEHRI (0.42 / 0.58) (Baseline TEHRI (0.42 / 0.58))
add_tehr(df, 1 - W_HH_BASE, W_HH_BASE, "EHRI_base")

# 3b. ReCiPe 等权 0.50 / 0.50 (ReCiPe equal weights 0.50 / 0.50)
add_tehr(df, 0.50, 0.50, "EHRI_recipe")

# 3c. 固定 ±20 % 敏感性场景 (Fixed ±20% sensitivity scenarios)
w_hh_low, w_hh_high = W_HH_BASE * (1 - PERTURB_FRAC), W_HH_BASE * (1 + PERTURB_FRAC)
add_tehr(df, 1 - w_hh_low,  w_hh_low,  "EHRI_down20")
add_tehr(df, 1 - w_hh_high, w_hh_high, "EHRI_up20")

# 4. 排名与 Kendall‑τ (固定场景) (Ranking and Kendall-τ (fixed scenarios)) --------------------------------------

def kendall_tau(series_a, series_b):
    return kendalltau(series_a, series_b, nan_policy="omit").correlation

rank_cols = ["EHRI_base", "EHRI_recipe", "EHRI_up20", "EHRI_down20"]
ranks_fixed = df[rank_cols].rank(ascending=False, method="min")

print("Kendall τ 与基准比较 (固定权重)")  # (Kendall τ compared to baseline (fixed weights))
for col in rank_cols[1:]:
    tau_val = kendall_tau(ranks_fixed["EHRI_base"], ranks_fixed[col])
    print(f"{col:12s}: {tau_val:.3f}")

tau_mc = []
rng = np.random.default_rng(seed=42)

# 添加额外权重场景 (Add additional weight scenarios)
add_tehr(df, 1 - 0.65,  0.65,  "EHRI_065")
ranks_fixed_065 = df['EHRI_065'].rank(ascending=False, method="min")
add_tehr(df, 1 - 0.6,  0.6,  "EHRI_06")
ranks_fixed_06 = df['EHRI_06'].rank(ascending=False, method="min")
add_tehr(df, 1 - 0.55,  0.55,  "EHRI_055")
ranks_fixed_055 = df['EHRI_055'].rank(ascending=False, method="min")
num065 = []
num06 = []
num055 = []

tehr_sim_sum = []
for w_hh in rng.uniform(low=w_hh_low, high=w_hh_high, size=N_SIM):
    tehr_sim = (1 - w_hh) * df["ERA_Max"] + w_hh * df["HHRA_Max"]
    tehr_sim_sum.append(tehr_sim)
    rank_sim = tehr_sim.rank(ascending=False, method="min")
    tau_mc.append(kendall_tau(ranks_fixed["EHRI_base"], rank_sim))

    # 计算大于特定阈值的数量 (Calculate count above specific thresholds)
    gt065 = np.sum(tehr_sim > 0.65)
    gt06 = np.sum(tehr_sim > 0.6)
    gt055 = np.sum(tehr_sim > 0.55)
    num065.append(kendall_tau(ranks_fixed_065, rank_sim))
    num06.append(kendall_tau(ranks_fixed_06, rank_sim))
    num055.append(kendall_tau(ranks_fixed_055, rank_sim))

tau_mc = np.asarray(tau_mc)
tehr_sim_sum = np.asarray(tehr_sim_sum)

num065 = np.asarray(num065)
num06 = np.asarray(num06)
num055 = np.asarray(num055)

# 6. Monte‑Carlo 结果汇总 (Monte Carlo results summary) ---------------------------------------------
summary_mc = {
    "N_sim": N_SIM,
    "tau_mean": tau_mc.mean(),  # Kendall τ 平均值 (Kendall τ mean)
    "tau_median": np.median(tau_mc),  # Kendall τ 中位数 (Kendall τ median)
    "tau_min": tau_mc.min(),  # Kendall τ 最小值 (Kendall τ minimum)
    "EHRI>0.55": np.sum(tehr_sim_sum > 0.55) / N_SIM,  # EHRI 大于 0.55 的比例 (Proportion of EHRI > 0.55)
    "EHRI>0.60": np.sum(tehr_sim_sum > 0.60) / N_SIM,  # EHRI 大于 0.60 的比例 (Proportion of EHRI > 0.60)
    "EHRI>0.65": np.sum(tehr_sim_sum > 0.65) / N_SIM,  # EHRI 大于 0.65 的比例 (Proportion of EHRI > 0.65)
    "tau_max": tau_mc.max(),  # Kendall τ 最大值 (Kendall τ maximum)
    "tau_2.5pct": np.percentile(tau_mc, 2.5),  # 2.5% 分位数 (2.5% percentile)
    "tau_97.5pct": np.percentile(tau_mc, 97.5),  # 97.5% 分位数 (97.5% percentile)
    "prop_material_shift": np.mean(np.abs(1 - tau_mc) > 0.05)  # 材料排名变化比例 (Proportion of material rank shifts)
}

print(f"\nMonte‑Carlo sensitivity (w_HH ±{PERTURB_FRAC*100:.0f} %, n={N_SIM})")  # (蒙特卡洛敏感性分析)
for k, v in summary_mc.items():
    if k != "N_sim":
        print(f"{k:20s}: {v:.3f}")

# 7. 导出结果 (Export results) ------------------------------------------------------------
# 保存固定权重排名 (Save fixed weight ranks)
ranks_fixed.add_suffix("_rank").to_excel("EHRI_fixed_ranks.xlsx", index=False)
# 保存 τ 分布 (Save τ distribution)
pd.DataFrame({"tau": tau_mc}).to_csv("tau_distribution.csv", index=False)
# 保存 τ 摘要 (Save τ summary)
pd.DataFrame(summary_mc, index=[0]).to_excel("tau_summary.xlsx", index=False)

# 保存大于阈值的数量 (Save counts above thresholds)
pd.DataFrame({
    "num065": num065, 
    "num06": num06, 
    "num055": num055 
}).to_csv("gt_num.csv", index=False)
import pandas as pd
import numpy as np
from scipy.stats import kendalltau

# ---------------------------------------------------------------------
# 参数区
EXCEL_FILE = "367EHRI Sensitivity Analysis.xlsx"   # 输入文件
SHEET_NAME = 'Sheet1'                 # 工作表索引/名称
N_SIM = 1000                    # Monte‑Carlo 迭代次数
W_HH_BASE = 0.58                # 基准人体健康权重 (UNEP‑GLAM)
PERTURB_FRAC = 0.20             # 浮动幅度 ±20 %
# ---------------------------------------------------------------------

# 1. 读取数据 -----------------------------------------------------------
df = pd.read_excel(EXCEL_FILE, sheet_name=SHEET_NAME)

# # 2. 计算保守人体健康概率 ---------------------------------------------
# df["P_HH"] = np.maximum(df["P_C"], df["P_NC"])

# 3. 辅助函数：计算并附加 EHRI 列 -------------------------------------

def add_tehr(df, w_eco: float, w_hh: float, col: str) -> None:
    df[col] = w_eco * df["ERA_Max"] + w_hh * df["HHRA_Max"]

# 3a. 基准 TEHRI (0.42 / 0.58)
add_tehr(df, 1 - W_HH_BASE, W_HH_BASE, "EHRI_base")

# 3b. ReCiPe 等权 0.50 / 0.50
add_tehr(df, 0.50, 0.50, "EHRI_recipe")

# 3c. 固定 ±20 % 敏感性场景
w_hh_low, w_hh_high = W_HH_BASE * (1 - PERTURB_FRAC), W_HH_BASE * (1 + PERTURB_FRAC)
add_tehr(df, 1 - w_hh_low,  w_hh_low,  "EHRI_down20")
add_tehr(df, 1 - w_hh_high, w_hh_high, "EHRI_up20")


# 4. 排名与 Kendall‑τ (固定场景) --------------------------------------

def kendall_tau(series_a, series_b):
    return kendalltau(series_a, series_b, nan_policy="omit").correlation

rank_cols = ["EHRI_base", "EHRI_recipe", "EHRI_up20", "EHRI_down20"]
ranks_fixed = df[rank_cols].rank(ascending=False, method="min")

print("Kendall τ 与基准比较 (固定权重)")
for col in rank_cols[1:]:
    tau_val = kendall_tau(ranks_fixed["EHRI_base"], ranks_fixed[col])
    print(f"{col:12s}: {tau_val:.3f}")

tau_mc = []
rng = np.random.default_rng(seed=42)

# 
add_tehr(df, 1 - 0.65,  0.65,  "EHRI_065")
ranks_fixed_065 = df['EHRI_065'].rank(ascending=False, method="min")
add_tehr(df, 1 - 0.6,  0.6,  "EHRI_06")
ranks_fixed_06 = df['EHRI_06'].rank(ascending=False, method="min")
add_tehr(df, 1 - 0.55,  0.55,  "EHRI_055")
ranks_fixed_055 = df['EHRI_055'].rank(ascending=False, method="min")
num065 = []
num06 = []
num055 = []

tehr_sim_sum = []
for w_hh in rng.uniform(low=w_hh_low, high=w_hh_high, size=N_SIM):
    tehr_sim = (1 - w_hh) * df["ERA_Max"] + w_hh * df["HHRA_Max"]
    tehr_sim_sum.append(tehr_sim)
    rank_sim = tehr_sim.rank(ascending=False, method="min")
    tau_mc.append(kendall_tau(ranks_fixed["EHRI_base"], rank_sim))

    # 
    # print('tehr_sim', tehr_sim)
    gt065 = np.sum(tehr_sim > 0.65)
    gt06 = np.sum(tehr_sim > 0.6)
    gt055 = np.sum(tehr_sim > 0.55)
    # gt055 = np.sum(tehr_sim < df["EHRI_055_045"])
    num065.append(kendall_tau(ranks_fixed_065, rank_sim))
    num06.append(kendall_tau(ranks_fixed_06, rank_sim))
    num055.append(kendall_tau(ranks_fixed_055, rank_sim))


tau_mc = np.asarray(tau_mc)
tehr_sim_sum = np.asarray(tehr_sim_sum)

num065 = np.asarray(num065)
num06 = np.asarray(num06)
num055 = np.asarray(num055)


# 6. Monte‑Carlo 结果汇总 ---------------------------------------------
summary_mc = {
    "N_sim": N_SIM,
    "tau_mean": tau_mc.mean(),
    "tau_median": np.median(tau_mc),
    "tau_min": tau_mc.min(),
    "EHRI>0.55": np.sum(tehr_sim_sum > 0.55) / N_SIM,
    "EHRI>0.60": np.sum(tehr_sim_sum > 0.60) / N_SIM,
    "EHRI>0.65": np.sum(tehr_sim_sum > 0.65) / N_SIM,
    "tau_max": tau_mc.max(),
    "tau_2.5pct": np.percentile(tau_mc, 2.5),
    "tau_97.5pct": np.percentile(tau_mc, 97.5),
    "prop_material_shift": np.mean(np.abs(1 - tau_mc) > 0.05)
}

print(f"\nMonte‑Carlo sensitivity (w_HH ±{PERTURB_FRAC*100:.0f} %, n={N_SIM})")
for k, v in summary_mc.items():
    if k != "N_sim":
        print(f"{k:20s}: {v:.3f}")

# 7. 导出结果 ------------------------------------------------------------
ranks_fixed.add_suffix("_rank").to_excel("EHRI_fixed_ranks.xlsx", index=False)
pd.DataFrame({"tau": tau_mc}).to_csv("tau_distribution.csv", index=False)
pd.DataFrame(summary_mc, index=[0]).to_excel("tau_summary.xlsx", index=False)

pd.DataFrame({
    "num065": num065, 
    "num06": num06, 
    "num055": num055 
}).to_csv("gt_num.csv", index=False)


Kendall τ 与基准比较 (固定权重)
EHRI_recipe : 0.932
EHRI_up20   : 0.913
EHRI_down20 : 0.899

Monte‑Carlo sensitivity (w_HH ±20 %, n=1000)
tau_mean            : 0.952
tau_median          : 0.950
tau_min             : 0.899
EHRI>0.55           : 339.558
EHRI>0.60           : 333.193
EHRI>0.65           : 330.582
tau_max             : 1.000
tau_2.5pct          : 0.904
tau_97.5pct         : 0.998
prop_material_shift : 0.498
